In [15]:
from utils import lime_explanation
import pandas as pd
import joblib
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline


In [21]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({
    'text': data.data,
    'target': data.target,
    'target_name': [data.target_names[i] for i in data.target]
})

class_names = df["target_name"].unique()

text_instance = df["text"].sample(n=1).values[0]

classifier = joblib.load("../02_classification/text_classifier.pkl")


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df["text"])

In [23]:
pipeline = make_pipeline(tfidf_vectorizer, classifier)

# Now classifier can handle it

def lime_explanations(model, class_names, text_instance):
    explainer = LimeTextExplainer(class_names=class_names)
    explanation = explainer.explain_instance(
        text_instance,
        model.predict_proba,     # must accept raw text
        num_features=10
    )
    return explanation

explanations = lime_explanations(pipeline, class_names, text_instance)
explanations.show_in_notebook()

ValueError: X has 134410 features, but SVC is expecting 50 features as input.